In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df = pd.read_csv('C:/Users/govindd1/Desktop/Code/VR-Project/ACC_data_from_different_subjects/E4_ACC_data.csv')
df.head()

,datatime,acc_x,acc_y,acc_z,label,subject_id,round
0,2021-10-27 16:19:17+00:00,-14,22,55,standing,2,1
1,2021-10-27 16:19:17.031250+00:00,-4,25,61,standing,2,1
2,2021-10-27 16:19:17.062500+00:00,-7,21,59,standing,2,1
3,2021-10-27 16:19:17.093750+00:00,-15,20,59,standing,2,1
4,2021-10-27 16:19:17.125000+00:00,-11,22,66,standing,2,1


In [3]:
df = df.drop('round', 1)
df = df.drop('datatime', 1)
df.head()

C:\Users\govindd1\AppData\Local\Temp/ipykernel_14412/532183799.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('round', 1)
C:\Users\govindd1\AppData\Local\Temp/ipykernel_14412/532183799.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('datatime', 1)


,acc_x,acc_y,acc_z,label,subject_id
0,-14,22,55,standing,2
1,-4,25,61,standing,2
2,-7,21,59,standing,2
3,-15,20,59,standing,2
4,-11,22,66,standing,2


In [5]:

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

activity_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(activity_name_mapping)

{'downstairs': 0, 'sitting': 1, 'standing': 2, 'upstairs': 3, 'walking': 4}


In [6]:
df

,acc_x,acc_y,acc_z,label,subject_id
0,-14,22,55,2,2
1,-4,25,61,2,2
2,-7,21,59,2,2
3,-15,20,59,2,2
4,-11,22,66,2,2
...,...,...,...,...,...
137401,-24,22,37,4,3
137402,-20,15,47,4,3
137403,-10,13,58,4,3
137404,-11,11,65,4,3


In [7]:
columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'label']
data = pd.DataFrame(data = df, columns = columns)
data.head()

,subject_id,acc_x,acc_y,acc_z,label
0,2,-14,22,55,2
1,2,-4,25,61,2
2,2,-7,21,59,2
3,2,-15,20,59,2
4,2,-11,22,66,2


In [19]:
#'downstairs': 0, 'sitting': 1, 'standing': 2, 'upstairs': 3, 'walking': 4#


In [27]:
# data = data.drop(['subject_id'], axis=1) 
data.head()

,acc_x,acc_y,acc_z,label
0,-14,22,55,2
1,-4,25,61,2
2,-7,21,59,2
3,-15,20,59,2
4,-11,22,66,2


In [32]:
subject_id = df.subject_id
subject_id

0         2
1         2
2         2
3         2
4         2
         ..
137401    3
137402    3
137403    3
137404    3
137405    3
Name: subject_id, Length: 137406, dtype: int64

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve,auc, roc_auc_score, make_scorer
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score


rf = RandomForestClassifier(random_state = 42) # random forest classifier as rf 
knn = KNeighborsClassifier() # K-Nearest Neighbor classifier as knn

In [44]:
feature = np.array(data.iloc[:,:-1]) # feature columns
target = np.array(data.iloc[:,-1]) # target column
groups = np.array(subject_id)

logo = LeaveOneGroupOut() # group wise splitting of train and test data
logo.get_n_splits(feature, target, groups)
logo.get_n_splits(groups=groups)

# empty lists for accuarcy and auc scores for each algorithm
accuracy_rf=[]
accuracy_knn=[]
auc_rf=[]
auc_knn=[]

subject_ID_avg = {}

for train,test in logo.split(feature,target,groups):
        
    rf_cv=rf.fit(feature[train],target[train])
    knn_cv=knn.fit(feature[train],target[train])

    y_pred_rf = rf_cv.predict(feature[test])
    # fpr, tpr, threshhold = roc_curve(target[test],y_pred_rf)
    # auc_RF = auc(fpr,tpr)
    accuracy_RF = accuracy_score(target[test],y_pred_rf)

    accuracy_rf.append(accuracy_RF)
    # auc_rf.append(auc_RF)

    y_pred_knn = knn_cv.predict(feature[test])
    # fpr, tpr, threshhold = roc_curve(target[test],y_pred_knn)
    # auc_KNN = auc(fpr,tpr)
    accuracy_KNN = accuracy_score(target[test],y_pred_knn)
    
    accuracy_knn.append(accuracy_KNN)
    # auc_knn.append(auc_KNN)

    subject_ID = groups[test][0]
    subject_ID_avg[str(subject_ID)] = {'accuracy_rf':accuracy_RF,
                                        # 'auc_rf':auc_RF,
                                        'accuracy_knn':accuracy_KNN
                                        # 'auc_knn':auc_KNN
                                        }
    

In [45]:
subject_ID_avg

{'1': {'accuracy_rf': 0.4264390034364261, 'accuracy_knn': 0.38319730813287517},
 '2': {'accuracy_rf': 0.226802675209352, 'accuracy_knn': 0.23107401787219692},
 '3': {'accuracy_rf': 0.2281515202607767, 'accuracy_knn': 0.241892879222166},
 '4': {'accuracy_rf': 0.22456633849038912,
  'accuracy_knn': 0.25996249413970934},
 '5': {'accuracy_rf': 0.3507021433850702, 'accuracy_knn': 0.3228851710004703}}